# In this notebook I will go through an approach to address class imbalance through undersampling the over represented data

In [48]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math 
from matplotlib.figure import Figure
pd.set_option('display.max_colwidth', None)
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
file=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')

In [4]:
details=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx')

In [5]:
details

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Data Description:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,ID,Customer ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Age,Customer's age in completed years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Experience,#years of professional experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Income,Annual income of the customer ($000),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
y=file['Personal Loan']

In [7]:
X=file.drop('Personal Loan', axis='columns')

In [8]:
X

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,1,0


## First approach: 
### Reduce sample of banking customers without personal loan and complete machine learnign model

In [14]:
file_0=file[file['Personal Loan']==0]
file_1=file[file['Personal Loan']==1]

In [15]:
len(file_0)

4520

In [16]:
len(file_1)

480

#### For the reduced sample approach, I will simply take a sample of 480 banking customers from the no personal loan dataframe

In [20]:
under_sample0=file_0.sample(480)

In [21]:
under_sample0

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
3529,3530,33,7,25,94132,4,1.000000,1,0,0,0,0,1,1
3831,3832,37,12,132,90089,2,3.900000,1,0,0,0,0,0,0
3965,3966,39,15,94,91941,2,1.900000,1,0,0,0,0,1,0
1219,1220,45,18,80,92407,3,2.666667,2,0,0,0,0,0,0
4789,4790,58,34,84,95120,4,1.600000,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,3476,54,30,13,92037,1,0.300000,1,0,0,0,0,0,0
1917,1918,62,32,53,96001,4,1.666667,3,142,0,0,0,0,0
4077,4078,26,0,71,92093,4,1.800000,2,0,0,1,0,1,0
2423,2424,50,25,82,91335,1,1.300000,3,0,0,0,0,1,0


In [23]:
file_1

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0
16,17,38,14,130,95010,4,4.7,3,134,1,0,0,0,0
18,19,46,21,193,91604,2,8.1,3,0,1,0,0,0,0
29,30,38,13,119,94104,1,3.3,2,0,1,0,1,1,1
38,39,42,18,141,94114,3,5.0,3,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,4884,38,13,129,92646,3,4.1,3,0,1,0,1,1,1
4927,4928,43,19,121,94720,1,0.7,2,0,1,0,1,1,1
4941,4942,28,4,112,90049,2,1.6,2,0,1,0,0,1,0
4962,4963,46,20,122,90065,3,3.0,3,0,1,0,1,1,1


Next, I will concatenate these two dataframes and apply machine learning modelling

In [29]:
file=pd.concat([file_1, under_sample0], axis=0)

In [30]:
file

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
9,10,34,9,180,93023,1,8.900000,3,0,1,0,0,0,0
16,17,38,14,130,95010,4,4.700000,3,134,1,0,0,0,0
18,19,46,21,193,91604,2,8.100000,3,0,1,0,0,0,0
29,30,38,13,119,94104,1,3.300000,2,0,1,0,1,1,1
38,39,42,18,141,94114,3,5.000000,3,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,3476,54,30,13,92037,1,0.300000,1,0,0,0,0,0,0
1917,1918,62,32,53,96001,4,1.666667,3,142,0,0,0,0,0
4077,4078,26,0,71,92093,4,1.800000,2,0,0,1,0,1,0
2423,2424,50,25,82,91335,1,1.300000,3,0,0,0,0,1,0


In [31]:
X=file.drop('Personal Loan', axis='columns')
y=file['Personal Loan']

In [32]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

!pip install category_encoders
%matplotlib inline

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [34]:

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (768, 13) (768,)
Test (192, 13) (192,)


In [35]:
print(X_train)

        ID  Age  Experience  Income  ZIP Code  Family     CCAvg  Education  \
4096  4097   38          14      49     90740       3  2.800000          1   
302    303   45          21     152     94550       2  7.500000          1   
3649  3650   53          29      85     92691       3  1.800000          2   
2815  2816   26           1      48     94019       3  2.600000          3   
2095  2096   47          21     174     94025       4  3.200000          3   
...    ...  ...         ...     ...       ...     ...       ...        ...   
3066  3067   63          33      40     91320       4  1.666667          3   
2715  2716   42          18      54     90089       1  1.800000          1   
3457  3458   55          31      91     94110       2  2.800000          1   
1063  1064   56          30      39     90024       3  1.400000          1   
1632  1633   31           5      93     95032       2  3.100000          2   

      Mortgage  Securities Account  CD Account  Online  CreditC

In [36]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [37]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [55]:


model = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=10000, subsample=0.5)

In [56]:
model.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=10000, subsample=0.5)

In [57]:
predictions=model.predict(X_test)


In [58]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 3264 to 339
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  192 non-null    int64  
 1   Age                 192 non-null    int64  
 2   Experience          192 non-null    int64  
 3   Income              192 non-null    int64  
 4   ZIP Code            192 non-null    int64  
 5   Family              192 non-null    int64  
 6   CCAvg               192 non-null    float64
 7   Education           192 non-null    int64  
 8   Mortgage            192 non-null    int64  
 9   Securities Account  192 non-null    int64  
 10  CD Account          192 non-null    int64  
 11  Online              192 non-null    int64  
 12  CreditCard          192 non-null    int64  
dtypes: float64(1), int64(12)
memory usage: 21.0 KB


In [59]:
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.99      0.96      0.97       100\n           1       0.96      0.99      0.97        92\n\n    accuracy                           0.97       192\n   macro avg       0.97      0.97      0.97       192\nweighted avg       0.97      0.97      0.97       192\n'

In [60]:

confusion_matrix(y_test,predictions)

array([[96,  4],
       [ 1, 91]])

In [61]:
accuracy_score(y_test,predictions)

0.9739583333333334

In [62]:
print('The Precision Score- What proportion of positive identifications was actually correct?: ' + str(100*precision_score(y_test,predictions))+'%')
print('The Recall Score - What proportion of actual positives was identified correctly?: ' + str(100*recall_score(y_test,predictions))+'%')

The Precision Score- What proportion of positive identifications was actually correct?: 95.78947368421052%
The Recall Score - What proportion of actual positives was identified correctly?: 98.91304347826086%


In [63]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=15)
print('Cross-Validation Accuracy Scores', scores.mean())
recall = cross_val_score(model, X_train, y_train, cv=15, scoring='recall')
print('Recall', recall.mean())
precision = cross_val_score(model, X_train, y_train, cv=15, scoring='precision')
print('Precision', precision.mean())

Cross-Validation Accuracy Scores 0.9674459527400704
Recall 0.9742564102564103
Precision 0.9688318433375905


In [47]:
# example of grid searching key hyperparameters for GradientBoostingClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# define models and parameters
model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='recall',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.5}
1.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.5}
1.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.7}
1.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 1.0}
0.978610 (0.018828) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}
0.976901 (0.022307) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
0.967431 (0.024736) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
0.973459 (0.021435) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.5}
0.971727 (0.023310) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.7}
0.978587 (0.021033) with: {'learning_rate': 0.001, 'm

# Accuracy: 98.45%
# Precision 94.33%
# Recall: 90.48%
